In [1]:
!pip install icrawler
!pip install ImageHash


In [2]:
from icrawler.builtin import BingImageCrawler, GoogleImageCrawler

from pathlib import Path


import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    


2.1.2
CUDA available: True
NVIDIA GeForce RTX 4070 Laptop GPU


In [8]:

root = Path("data/raw") 
path = root

(root / "manga").mkdir(parents=True, exist_ok=True)
(root / "manhwa").mkdir(parents=True, exist_ok=True) 
(root / "manhua").mkdir(parents=True, exist_ok=True)

In [9]:
queries = {
    "manga": [
        'manga page -manhwa -manhua',
        'manga panel site:ja.wikipedia.org'
    ],
    "manhwa": [
        'manhwa panel -manhua -manga',
        'webtoon panel -manga -manhua',
        '웹툰 패널 site:comic.naver.com',
        'webtoon panel site:webtoons.com -manga -manhua'
    ],
    "manhua": [
        'manhua panel -manhwa -manga',
        '国漫 漫画 分镜 -webtoon',
        '中国 漫画 分镜 -webtoon',
        'manhua site:bilibili.com -webtoon -manhwa -manga'
    ],
}

In [10]:
def crawl_images(label, n_per_query): 
    save_path = root / label
    if save_path.exists() and any(save_path.iterdir()):
        print(f"Images for '{label}' already exist. Skipping download.")
        return
        
    for q in queries[label]: 
        crawler = BingImageCrawler(storage={'root_dir': str(root/label)})
        crawler.crawl(
            keyword=q,
            max_num=n_per_query,
            min_size=(256, 256),  
            file_idx_offset='auto'
        )

In [11]:
for label in queries:
    crawl_images(label, n_per_query=30)

Images for 'manga' already exist. Skipping download.
Images for 'manhwa' already exist. Skipping download.
Images for 'manhua' already exist. Skipping download.


In [37]:
from PIL import Image, UnidentifiedImageError
import imagehash, os
from fastai.vision.all import *


dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460),         # presize
    batch_tfms=[*aug_transforms(size=224), Normalize.from_stats(*imagenet_stats)]
).dataloaders(path, bs=32)

learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(5)   

epoch,train_loss,valid_loss,accuracy,time
0,1.557265,0.966833,0.666667,00:36


epoch,train_loss,valid_loss,accuracy,time
0,0.934501,0.874977,0.666667,00:34
1,0.832181,0.679173,0.801282,00:34
2,0.674094,0.880799,0.782051,00:35
3,0.565812,0.810484,0.782051,00:35
4,0.467886,0.802520,0.807692,00:32


In [30]:
classes = list(learn.dls.vocab)

print(classes)

['manga', 'manhua', 'manhwa']


In [35]:
im = PILImage.create("data/test/test2_manhwa.jpeg")

In [36]:
predicted_class,_,probs = learn.predict(im)
# is_bird,_,probs = learn.predict(PILImage.create('forest000001.png'))
print(f"This is a: {predicted_class}.")
predicted_idx = classes.index(predicted_class)
# print(f"Probability it's a {predicted_class}: {probs[predicted_idx]:.4f}")
for idx, ele in enumerate(classes):
    print(f"Probability it's a {ele}: {probs[idx]:.4f}")

This is a: manhua.
Probability it's a manga: 0.1245
Probability it's a manhua: 0.6890
Probability it's a manhwa: 0.1865
